# Huffman Coding

In [1]:
#include <iostream>
#include <string>
#include <queue>       // std::priority_queue
#include <vector>
#include <functional>  // std::greater
#include <unordered_map>
#include <vector>

class HeapNode
{
public: 
    const HeapNode* left = nullptr;
    const HeapNode* right = nullptr;
    int freq;
    char ch;
    
    HeapNode(int freq, char ch, const HeapNode* left, const HeapNode* right) 
        : freq(freq), ch(ch), left(left), right(right) {}
    
    void print() const { 
        std::cout << "(" << ch << ", " << freq << ")" << std::endl; 
    }
    friend bool operator>(const HeapNode& a, const HeapNode& b) { 
        return a.freq > b.freq; 
    }
};

std::string message(R"(In computer science and information theory, a Huffman code is a particular type of optimal prefix code that is commonly used for lossless data compression. The process of finding or using such a code proceeds by means of Huffman coding, an algorithm developed by David A. Huffman while he was a Sc.D. student at MIT, and published in the 1952 paper "A Method for the Construction of Minimum-Redundancy Codes".)");

std::unordered_map<char, int> freq;
for (const char& c : message)
    freq[c]++;

std::vector<HeapNode> nodes;
for (auto tuple : freq){
    char letter = tuple.first;
    int frequency = tuple.second;
    nodes.push_back(HeapNode(frequency, letter, nullptr, nullptr));
}

std::priority_queue<HeapNode, std::vector<HeapNode>, std::greater<HeapNode>> q(std::greater<HeapNode>(), nodes);

while(q.size() > 1) 
{
    const HeapNode* left = &(q.top());
    q.pop();
    
    const HeapNode* right = &(q.top());
    q.pop();
    
    q.push(HeapNode(left->freq + right->freq, '\0', left, right));
}